# Úvod do autoenkodérov

Na dnešnom cvičení sa oboznámime s novým typom neurónových sietí - autoenkodérmi. Autoenkodéry sú neurónové siete, ktoré slúžia na kompresiu údajov, avšak oproti tradičným metódam kompresie nepotrebujeme ich vopred naprogramovať. Ďalšou ich vlastnosťou je že sú špecificky určené na kompresiu údajov podobných ako dáta, na ktorých bola sieť natrénovaná. Podobne ako ďalšie metódy kompresie (napr. mp3, jpeg), aj autoenkodéry robia  stratovú kompresiu.

Z topologického hľadiska autoenkodéry sú zvyčajne symetrické (síce nie je to nevyhnutné, v praxi takéto siete sa jednoduchšie trénujú). Cieľom autoenkodérov je kopírovať vstup na výstupe tak, že skryté vrstvy majú menej neurónov ako vstup, resp. výstup. Autoenkodéry teda majú dve časti, prvá (enkodér) má za úlohu skomprimovať údaje, kým druhá časť (dekodér) reprodukuje vstup zo zakódovaného tvaru.

![](resources/lab09/autoencoder.png)

Dnes si vytvoríme dva jednoduché autoenkodéry pomocou knižnice Keras, najprv bez konvolúcie. Kostru riešenia nájdete [tu](resources/lab09/lab09.py).

## 1. Načítanie potrebných knižníc

V riešení využijeme už známe knižnice `keras`, `numpy` a `matplotlib` pre vizualizáciu fungovania siete:

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np

## 2. Vizualizácia výsledkov

V kóde máte pripravenú funkciu `plot_results`, ktorá slúži na vizualizáciu fungovania siete. Pomocou knižnice `matplotlib` zobrazíme niekoľko ukážkových vstupov, ich zakódovaný tvar a rekonštruovaný obraz.

In [ ]:
def plot_results(x_test, encoded_imgs, decoded_imgs, n=10):
    plt.figure(figsize=(40, 4))
    for i in range(n):
        # display original
        ax = plt.subplot(3, n, i + 1)
        plt.imshow(x_test[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display encoded
        ax = plt.subplot(3, n, i + 1 + n)
        plt.imshow(encoded_imgs[i].reshape(8, 4))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(3, n, i + 1 + n * 2)
        plt.imshow(decoded_imgs[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

## 3. Príprava údajov

Rovnako ako pri predošlých sieťach, aj dnes použijeme dataset rukou písaných číslic MNIST, keďže ale nepoužijeme konvolúciu, tieto obrázky potrebujeme reprezentovať ako jedrozmerné vektory. Pred tým ale pixely normalizujeme, t.j. pretypujeme ich na numpyovský float a hodnoty od 0 po 255 namapujeme do intervalu 0 až 1.

Všimnite si, že z datasetu nenačítame labely (nahradíme ich znakom `_`). Dôvodom je, že autoenkodéry by mali mať na výstupe rovnakú hodnotu ako na vstupe, teda stačia nám vstupné údaje z datasetu.

In [ ]:
(mnist_train, _), (mnist_test, _) = mnist.load_data()

x_train = # TODO: normalize to range 0 to 1
x_test = # TODO: normalize to range 0 to 1
x_train = # TODO: reshape to a list of one-dimensional vectors
x_test = # TODO: reshape to a list of one-dimensional vectors

## 4. Shallow autoencoder

V ďalšom kroku zadefinujeme najjednoduchší možní autoenkodér: autoenkodér s jednou skrytou vsrtvou.

### 4.1. Definícia vrstiev
Keďže vrstvy chceme využit niekoľkokrát, namiesto toho, aby sme najprv vytvorili model a postupne sme doňho pridávali vrstvy, teraz najprv zadefinujeme vrstvy, ktoré spojíme pomocou parametrov v tvare:

In [ ]:
dalsia_vrstva = Typ_vrstvy(parametre)(predosla_vrstva)

In [ ]:
input_img = # TODO: define input layer
encoded = # TODO: define hidden layer
decoded = # TODO: define output layer

### 4.2. Definícia modelov
Ak už máme vrstvy zadefinované, môžeme pristúpiť k definícii modelov. V tomto kroku zadefinujeme tri modely: celkový autoenkodér, enkodér, a dekodér. Modely autoenkodéra a enkodéra vieme zadefinovať na základe ich vstupu a výstupu v tvare:

In [ ]:
model_object = Model(vstupna_vrstva, vystupna_vrstva)

In [ ]:
autoencoder = # TODO: define autoencoder
encoder = # TODO: define encoder

Definícia modelu čistého dekodéra je menej priamočiara. Jeho vstupom je síce výstup enkodéra, ale nevieme priamo použiť vrstvy z autoenkodéra, keďže tie sú navzájom prepojené. Namiesto toho vytvoríme novú vstupnú vrstvu a následne pridáme ďalšie potrebné vrstvy (v tomto prípade iba poslednú vrstvu autoenkodéra).

In [ ]:
encoded_input = # TODO: define input to decoder
decoder_layer = # TODO: define further layer of decoder
decoder = # TODO: define decoder

### 4.3. Trénovanie autoenkodéra

Neostáva nám nič iné, len natrénovať našu sieť. Pridajte chýbajúce parametre do volaní funkcií nasledovne:
* `optimizer` - napr. adadelta
* `loss` - napr. binary_crossentropy
* `input` - vstup autoenkodéra
* `output` - očakávaný výstup autoenkodéra
* `epochs` - napr. 25
* `batch_size` - napr. 256
* `shuffle` - nastavte na True pre náhodné poradie v trénovacej množine
* `verbose` - nastavte na 2 pre jednoduchší výpis progresu

In [ ]:
autoencoder.compile(# TODO: insert parameters: optimizer, loss
    )
autoencoder.fit(# TODO: insert parameters: input, output, epochs, batch_size, shuffle
    )

### 4.4. Testovanie autoenkodéra

Ak už máme autoenkodér natrénovaný, môžeme vizualizovať jeho funkčnosť. Zavolajte funkciu `predict` nad testovacou množinou. Keďže chceme vizualizovať vstup, zakódovanú reprezentáciu, aj výstup autoenkodéra, funkciu potrebujeme zavolať aj pre enkodér aj pre samotný autoenkodér. Ak chcete testovať zvlášť aj dekodér, upravte kód podľa potreby.

In [ ]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = autoencoder.predict(x_test)

plot_results(x_test, encoded_imgs, decoded_imgs)

Zobrazia sa Vám tri riadky obrázkov, kde prvý riadok je vstup, druhý je zakódovaný tvar, a posledný riadok obsahuje rekonštruované obrázky, t.j. výstup autoenkodéra, napr.

![](resources/lab09/autoencoder-shallow.jpg)

## 5. Deep autoencoder

V ďalšom kroku rozšírime našu sieť pridávaním ďalších vrstiev (samotná topológia je na Vás, nižšie máte uvedený príklad). Odporúčame Vám použiť aktivačnú funkciu `relu` až na poslednú vrstvu, kde sa použije `sigmoid`.

```
Layer (type)                 Output Shape              Param #   
_________________________________________________________________
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_5 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_6 (Dense)              (None, 784)               101136    
_________________________________________________________________
Total params: 222,384
Trainable params: 222,384
Non-trainable params: 0
```

### 5.1. Definícia vrstiev

Zadefinujte jednotlivé vrstvy hlbokej neurónovej siete podobne ako v bode 4.1.

In [ ]:
input_img = # TODO: define input layer
encoded_1 = # TODO: define next layer; activation relu
encoded_2 = # TODO: define next layer; activation relu
encoded_3 = # TODO: define next layer; activation relu

decoded_1 = # TODO: define next layer; activation relu
decoded_2 = # TODO: define next layer; activation relu
decoded_3 = # TODO: define next layer; activation sigmoid

### 5. 2. Definícia modelov

Podobne ako v bode 4.2., zase zadefinujeme tri modely: celý autoenkodér, iba enkodér a iba dekodér. Pri vytváraní modelu dekodéra dbajte na to, aby ste pridali všetky potrebné vrstvy.

In [ ]:
autoencoder = # TODO: define autoencoder

encoder = # TODO: define encoder

encoded_input = # TODO: define input to decoder
decoder_layer = # TODO: define next layer of the decoder
decoder_layer = # TODO: define next layer of the decoder
decoder_layer = # TODO: define next layer of the decoder
decoder = # TODO: define decoder

### 5. 3. Trénovanie autoenkodéra

V ďalšom kroku natrénujeme hlboký autoenkodér rovnako ako v bode 4.3.

In [ ]:
autoencoder.compile(# TODO: insert parameters: optimizer, loss
    )
autoencoder.fit(# TODO: insert parameters: input, output, epochs, batch_size, shuffle
    )

### 5. 4. Testovanie autoenkodéra

Vizualizácia bude mať rovnaký tvar ako v bode 4.4. V tomto bode ale pre ukážku nepoužijeme autoenkodér, ale zvlášť enkodér a následne dekodér. Môžete dostať výsledok napr.:

![](resources/lab09/autoencoder-deep.jpg)

In [ ]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

plot_results(x_test, encoded_imgs, decoded_imgs)

## 6. Autoenkodér s konvolúciou

Predošlé siete obsahovali iba tradičné plne prepojené vrstvy, v tomto kroku zadefinujeme a natrénujeme autoenkodér s konvolučnými vrstvami a pritom si vysvetlíme, ako funguje *upsampling* (opak *pooling*u). Kostru riešenia nájdete [na tomto odkaze](resources/lab09/lab09_conv.py).

### 6. 1. Načítanie potrebných knižníc

Novinkou medzi použitými modulmi a triedami je vrstva `UpSampling2D`, ktorý slúži na "dekonvolúciu", teda o rozšírenie rozmerov obrazu.

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

### 6.2. Vizualizácia výsledkov

Aj v tomto prípade chceme otestovať fungovanie nášho autoenkodéra a porovnať rekonštruovaný obraz s originálom. Na tento účel sme zadefinovali zjednodušnú funkciu `plot_results`, ktorý zobrazí ukážkové vstupy a výstupy autoenkodéra (odstránili sme vizualizáciu zakódovaného tvaru).

In [ ]:
def plot_results(x_test, decoded_imgs, n=10):
    plt.figure(figsize=(40, 4))
    for i in range(n):
        # display original
        ax = plt.subplot(3, n, i + 1)
        plt.imshow(x_test[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(3, n, i + 1 + n * 2)
        plt.imshow(decoded_imgs[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

### 6. 3. Príprava údajov

Ďalším krokom v riešení je úprava údajov z datasetu MNIST tak, aby sme ich vedeli použiť pre našu sieť. Vykonáme tie isté kroky, ako v predošlých úlohách, teda hodnoty najprv pretypujeme na *float* a normalizujeme ich na interval 0 až 1. Labely z datasetu stále nepotrebujeme, použijeme iba vstupné hodnoty.

Ďalej potrebujeme trénovaciu aj testovaciu množinu upraviť do tvaru, s ktorým vie naša sieť pracovať. Keďže samotný dataset už obsahuje fotky s rozmerom *28x28*, nepotrebujeme vykonať veľké zmeny, dataset iba rozšíreme o jednu dimenziu.

In [ ]:
x_train = # TODO: make type float32, normalize to interval 0 to 1
x_test = # TODO: make type float32, normalize to interval 0 to 1
x_train = # TODO: reshape
x_test = # TODO: reshape

### 6.4. Definícia autoenkodéra

V ďalšom kroku zadefinujeme náš autoenkodér. Vstupná vrstva je definovaná trénovacou množinou, ale okrem toho môžete navrhnúť ľubovoľnú topológiu. Kostra riešenia je ale napísaná pre nasledujúcu štruktúru, s kernelom *3x3*:

```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
_________________________________________________________________
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 8)           0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 8)           584       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 8, 8, 8)           0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 8)           584       
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 16)        1168      
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 28, 28, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 1)         145       
_________________________________________________________________
Total params: 4,385
Trainable params: 4,385
Non-trainable params: 0
```

In [ ]:
input_img = # define input layer

encoded_1 = # define first convolution, activation relu, padding same
encoded_2 = # define max pooling layer, padding same
encoded_3 = # define second convolution, activation relu, padding same
encoded_4 = # define max pooling layer, padding same
encoded_5 = # define third convolution, activation relu, padding same
encoded = # define max pooling layer, padding same

decoded_1 = # define first decoding convolution, activation relu, padding same
decoded_2 = # define first upsampling layer
decoded_3 = # define second decoding convolution, activation relu, padding same
decoded_4 = # define second upsampling layer
decoded_5 = # define third decoding convolution, activation relu
decoded_6 = # define third upsampling layer
decoded = # define final decoding convolution, activation sigmoid, padding same

autoencoder = Model(input_img, decoded)

### 6.5. Trénovanie a testovanie modelu

Keď už máme sieť zadefinovanú, neostáva nám nič iné ako sieť natrénovať a otestovať. Keďže trénovanie môže trvať dlhší čas, našu sieť otestujeme po každej epoche, práve preto zadefinujeme cyklus, ktorý v každej iterácii vykoná iba jednu epochu a následne zobrazí ukážkové vstupy a výstupy pomocou volania funkciue `plot_results`.

Vo funkcii `fit` nastavte nasledujúce parametre:
* `input` - vstup autoenkodéra
* `output` - očakávaný výstup autoenkodéra
* `epochs` - 1
* `batch_size` - napr. 128
* `shuffle` - nastavte na True pre náhodné poradie v trénovacej množine
* `verbose` - nastavte na 1

In [ ]:
autoencoder.compile(# optimizer adadelta, loss function binary_crossentropy
    )

EPOCH_NO = 25
for x in range(EPOCH_NO):
    autoencoder.fit(# define parameters
        )

    decoded_imgs = autoencoder.predict(x_test)

    plot_results(x_test, decoded_imgs)